In [ ]:
pip install yfinance

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import yfinance as yf
from pandas_datareader import data as pdr
yf.pdr_override()

import keras
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input, Activation, concatenate, Dropout
from sklearn.metrics import mean_squared_error, r2_score
import plotly.graph_objects as go


In [ ]:
stock_name = "GOOG"
dataset=pdr.get_data_yahoo(stock_name, start='2015-01-01', end='2018-03-08').reset_index()
dataset.index = dataset['Date']

print("Number of rows and columns:", dataset.shape)
dataset.head(5)

[*********************100%***********************]  1 of 1 completed
Number of rows and columns: (800, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,,
2015-01-02,2015-01-02,527.561584,529.815369,522.665039,523.373108,523.373108,1447563
2015-01-05,2015-01-05,521.827332,522.894409,511.655243,512.463013,512.463013,2059840
2015-01-06,2015-01-06,513.589966,514.761719,499.678131,500.585632,500.585632,2899940
2015-01-07,2015-01-07,505.611847,505.855164,498.281952,499.727997,499.727997,2065054
2015-01-08,2015-01-08,496.626526,502.101471,489.655640,501.303680,501.303680,3353582


In [ ]:
close_data = dataset['Close'].values
close_data = close_data.reshape((-1,1))

scaler = MinMaxScaler(feature_range=(0, 1))
close_data = scaler.fit_transform(close_data)

In [ ]:

split_percent = 0.70
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = dataset['Date'][:split]
date_test = dataset['Date'][split:]


In [ ]:
look_back = 5

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=40)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=5)

In [ ]:
model = Sequential()
model.add(
    LSTM(700,
        activation='relu',
        input_shape=(look_back,1)
        )
)

model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 20
model.fit(train_generator,epochs=num_epochs, verbose=1)

Epoch 1/20
14/14 [==============================] - 3s 117ms/step - loss: 0.0491
Epoch 2/20
14/14 [==============================] - 1s 102ms/step - loss: 0.0043
Epoch 3/20
14/14 [==============================] - 1s 104ms/step - loss: 0.0016
Epoch 4/20
14/14 [==============================] - 3s 188ms/step - loss: 5.4604e-04
Epoch 5/20
14/14 [==============================] - 2s 165ms/step - loss: 5.9345e-04
Epoch 6/20
14/14 [==============================] - 3s 183ms/step - loss: 8.7073e-04
Epoch 7/20
14/14 [==============================] - 3s 220ms/step - loss: 5.6773e-04
Epoch 8/20
14/14 [==============================] - 3s 192ms/step - loss: 5.5179e-04
Epoch 9/20
14/14 [==============================] - 1s 102ms/step - loss: 5.5127e-04
Epoch 10/20
14/14 [==============================] - 1s 102ms/step - loss: 5.6386e-04
Epoch 11/20
14/14 [==============================] - 1s 103ms/step - loss: 5.8223e-04
Epoch 12/20
14/14 [==============================] - 1s 103ms/step - loss: 

In [ ]:
prediction = model.predict(test_generator)

In [ ]:

def predict(num_prediction, model):
    prediction_list = close_test[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list
    
def predict_dates(num_prediction):
    last_date = dataset['Date'].values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1).tolist()
    return prediction_dates

num_prediction = 40
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)



In [ ]:
#close_test=close_test[:-look_back]

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))

trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Predicted Data'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Test Data'
)
trace4 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode='lines',
    name = 'Future Data'
)
layout = go.Layout(
    title = stock_name,
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[trace1, trace2, trace3, trace4], layout=layout)
fig.show()

In [ ]:
close_test2=close_test[:-look_back]
RNNmse = mean_squared_error(close_test2,prediction)
from math import sqrt
RNNrmse = sqrt(RNNmse)
RNNr2Score = r2_score(close_test2,prediction)

RNNmse,RNNrmse,RNNr2Score

(0.000398602238690598, 0.01996502538667552, 0.973290142338298)